# Clean and link filings



## 0. Imports and functions

In [1]:
from tabula import read_pdf
import os
import pandas as pd
import pickle
import re
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## profiling
import time

## plotting
from plotnine import *



## first, clean case type
def process_type(one_row):
    
    ## some dates so convert to string
    one_string = str(one_row)
    
    ## clean for expedited discipline
    clean_exp_1 = re.sub(r'(Exped(i)?(t)?(e)?|Discip)\s+', r'\1', one_string)
    clean_exp_2 = re.sub(r'(Exped(i)?(t)?(e)?|Discip)\s+', r'\1', clean_exp_1)
    
    ## clean for lea
    clean_lea = re.sub(r'(Aga(i)?(n)?)\s+', r'\1', clean_exp_2)
    clean_lea_ret = clean_lea.lower()
    
    return(clean_lea_ret)

def process_schoolname(one_name):
    
    ## uppercase
    name_str = str(one_name)
    name_upper = name_str.upper()

    ## clean up schools
    clean_school= re.sub(r'(SCHOO)\s+', r'\1', name_upper)
    clean_middle = re.sub(r'(MIDD)\s+', r'\1', clean_school)
    clean_ed = re.sub(r'(EDUCAT)\s+', r'\1', clean_middle)
    
    ## concat whitespace
    replace_middle = re.sub(r'M(\s)?I(\s)?D(\s)?D(\s)?L(\s)?E', r"MIDDLE", clean_ed)
    replace_elem = re.sub(r'E(\s)?L(\s)?E(\s)?M(\s)?E(\s)?N(\s)?T(\s)?A(\s)?R(\s)?Y', r"ELEMENTARY", replace_middle)
    replace_school = re.sub(r'SCHOOI', "SCHOOL", replace_elem)
    replace_campus = re.sub(r'C(\s)?A(\s)?M(\s)?P(\s)?U(\s)?S', r"CAMPUS", replace_school)
    replace_education = re.sub(r'E(\s)?D(\s)?U(\s)?C(\s)?A(\s)?T(\s)?I(\s)?O(\s)?N', r"EDUCATION", 
                               replace_campus)
    
    ## 

    return(replace_education)

def replace_schooltype(one_string):
    
    es = re.sub(r'ES$|ELEMENTARY$', r'ELEMENTARY SCHOOL', one_string)
    ec = re.sub(r'EC$', r'ELEMENTARY CAMPUS', es)
    ms = re.sub(r'MS$|MIDDLE$', r'MIDDLE SCHOOL', ec)
    hs = re.sub(r'HS$|HIGH$', r'HIGH SCHOOL', ms)
    
    return(hs)


    

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

from sklearn.feature_extraction.text import TfidfVectorizer


def find_fuzzy_namematches(one_name: str, all_names: list, 
                           score_cutoff):
    
    ## extract matches above cutoff
    all_abovecutoff = process.extractBests(one_name, all_names, score_cutoff = score_cutoff,
                                          limit = 1)
    
    ## make into a dataframe (will thus only capture ones with matches)
    all_abovecutoff_df = pd.DataFrame(list(all_abovecutoff), columns = ['matched_name', 'score'])
    all_abovecutoff_df['original_name'] = one_name
    return(all_abovecutoff_df)

## resource-- package installation issues: https://bergvca.github.io/2017/10/14/super-fast-string-matching.html



def replace_missing_nces(one_val):
    
    if one_val.isdigit():
        final_val = one_val
    else:
        final_val = np.nan
    return(final_val)

def aggregate_crdc(var_col, value_col, 
                  data,
                  id_col = "school_name",
                  year_chosen = 2013, format = "long"):
    
    
    ## shape from long to wide
    if(format == "long"):
        df_wide = pd.pivot_table(data.loc[data.year == year_chosen,
                    [id_col,
                    var_col, 
                    value_col]],
                    index  = id_col, 
                    values = value_col,
                    columns = var_col).reset_index()

    
    else:
        df_wide = data.loc[data.year == year_chosen].copy()
        
    ## standardize columns
    df_wide.columns = [re.sub("\s+", "_", col.upper()) 
                           for col in df_wide.columns]
        
    ## generate rates
    sub_cols = set(df_wide.columns).difference(["SCHOOL_NAME", "TOTAL", 'YEAR'])
    
    ## 
    for col in sub_cols:
        df_wide[col] = pd.to_numeric(df_wide[col])
        df_wide['TOTAL'] = pd.to_numeric(df_wide['TOTAL'])
        df_wide['{}_rate'.format(col)] = df_wide[col]/df_wide['TOTAL']
    
    ## return
    return(df_wide)
    
def aggregate_nces(var_pattern, varname_clean, id_col, 
                       cc_data_merged,
                      enrollment_vars, base_name_raw):
    
    dem_vars = [col for col in cc_data_merged if var_pattern in col]
    long_df = pd.melt(cc_data_merged[dem_vars + enrollment_vars + id_col],
                       id_vars = id_col)
    
    ## create year versus dem col
    long_df['clean_value_1'] = [replace_missing_nces(val) for val in long_df.value]
    long_df['clean_value'] = pd.to_numeric(long_df.clean_value_1)
    long_df['which_var'] = long_df.variable.astype(str).str.replace("\\_20.*", "", regex = True)
    replace_pattern = "|".join(long_df.which_var.unique())
    long_df['which_year'] = [re.sub(replace_pattern, "", one_var) for one_var in long_df.variable]
    long_toagg = long_df[id_col + ['which_var', 'which_year',
                                           'clean_value']].reset_index()

    ## do the aggregation 
    index_cols = id_col + ["which_year"]
    df_wide = long_toagg.pivot_table(index  = index_cols,
                                             values = 'clean_value',
                                             columns = 'which_var').reset_index()

    ## do the division
    rate_varname = varname_clean + '_rate'
    df_wide[rate_varname] = df_wide[var_pattern]/df_wide[base_name_raw]
    #print(df_wide.head())
    
    ## return cleaned data
    return(df_wide[id_col + [rate_varname] + ['which_year']])
        

/Users/rebeccajohnson/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 1. Load and do prelim cleaning of filings data

In [2]:
dc_filings = pd.read_excel("../../raw_input/dc/dc_alternate_extraction.xlsx")
dc_filings.head()


,date_filed,case_no,stud_homeschool,stud_attendschool,case_type,issue_case_type,against_osse_name,against_dcps_name,against_charter_name,type_closingorder,date_closed_orderissued
0,1/3/2012,2012-0002,NaN,Prospect LC,Against LEA,Against LEA,NaN,prospect LC,NaN,Order,1/27/2012
1,1/3/2012,2012-0003,NaN,Elsie Whitlow Stokes Community Freedom PCS,Against LEA,Against LEA,NaN,NaN,NaN,Order,1/19/2012
2,1/4/2012,2012-0006,NaN,Cesar Chavez PCS,Against LEA,Against LEA,NaN,NaN,Cesar Chavez PCS,Order,5/8/2012
3,1/4/2012,2012-0005,NaN,Columbia Heights Education Center High School,Against LEA,Against LEA,NaN,Columbia Heights Education Center High School,NaN,Order,2/28/2012
4,1/5/2012,2012-0007,Johnson MS,Johnson MS,Against LEA,Against LEA,NaN,Johnson MS,NaN,HOD,1/20/2012


## 1.1 Cleaning case type

In [3]:


dc_filings['casetype_clean_init'] = [process_type(one_type) for one_type in dc_filings.case_type.tolist()]
dc_filings.casetype_clean_init.value_counts()


against lea                    3117
expedited discipline            272
nan                              61
expedited non discipline         50
by lea                           30
aga•nst lea                      14
against sea                      10
expedited non                     5
agailea                           5
expedited discipline by lea       1
expedited non discioline          1
Name: casetype_clean_init, dtype: int64

In [4]:
dc_filings['casetype_isdigits'] = ["digits" if re.match(r'[0-9]+', one_str) is not None  else "no_digits" 
        for one_str in dc_filings.casetype_clean_init]

pd.set_option('display.max_rows', None)

In [5]:
casetypes_conds = [dc_filings.casetype_clean_init.str.contains("expedited non"),
                   (dc_filings.casetype_clean_init.str.contains("discipline")) | 
                   (dc_filings.casetype_clean_init.str.contains("expedited disc")),
                   (dc_filings.casetype_clean_init.str.contains("lea")) & 
                    (dc_filings.casetype_clean_init != "by lea"),
                   dc_filings.casetype_clean_init == "by lea",
                   dc_filings.casetype_clean_init.str.contains("against se")]
casetypes_codeto = ['Expedited non-discipline', 'Expedited Discipline', 'Against LEA', 'Against SEA', 
                   'By LEA']

assert len(casetypes_conds) == len(casetypes_codeto)

## apply
dc_filings['derived_casetype_final'] = np.select(casetypes_conds,
                                        casetypes_codeto,
                                        default= 'Other/failed to parse')
dc_filings.derived_casetype_final.value_counts()


Against LEA                 3136
Expedited Discipline         273
Other/failed to parse         61
Expedited non-discipline      56
Against SEA                   30
By LEA                        10
Name: derived_casetype_final, dtype: int64

## 1.2 Cleaning year 

In [6]:
dc_filings['date_filed_temp'] = np.where(dc_filings.case_no == "2015-0083", 
                                          "03/11/2015",
                                np.where(dc_filings.case_no == "2016-0287",
                                            "12/06/2016",
                                           dc_filings.date_filed)) # manually fixing 1 where month got cutoff
dc_filings['derived_date_filed'] = pd.to_datetime(dc_filings.date_filed_temp, errors = "coerce")

dc_filings['derived_date_closed'] = pd.to_datetime(dc_filings.date_closed_orderissued, errors = "coerce")


## construct indicator for duration
dc_filings['derived_daysbt_fileclosed'] = np.where(~dc_filings.derived_date_closed.isnull(),
                (dc_filings['derived_date_closed'] - dc_filings['derived_date_filed']).dt.days,
                np.nan)
#dc_filings.derived_daysbt_fileclosed.value_counts()

### maybe add business day counts and school years - np.busday_count('2018-04-10', '2018-04-11') 

## 2. Merge in district demographic data

### 2.1 Create name-nces ID crosswalk using common core data

In [7]:
## 75-col limit in export-- first 75 cols
cc_data_1 = pd.read_csv("../../raw_input/dc/dc_ccd.csv")

## 
cc_data_2 = pd.read_csv("../../raw_input/dc/dc_ccd_pull2.csv",
                       encoding= 'unicode_escape')

## find overlapping cols
cc_data_1_topull = cc_data_1.columns.difference(cc_data_2.columns).tolist() + \
                    ["School Name", 
                     "School ID - NCES Assigned [Public School] Latest available year"]

In [9]:
## merge excluding
cc_data_merged = pd.merge(cc_data_1[cc_data_1_topull], 
                          cc_data_2[[col for col in cc_data_2.columns if 
                            "School ID - NCES Assigned [Public School] Latest available year" not in col]], 
                          on = ["School Name"],
                         how = "left")

#cc_data_merged[[col for col in cc_data_merged.columns if 'ID' in col] + ['School Name']].sample(n = 10)


In [10]:
cc_cleancols = [re.sub("\\s+|\\[|\\]|\\(|\\)", "_", x).upper() for x in cc_data_merged.columns]
cc_data_merged.columns = cc_cleancols


In [11]:
## create crosswalk to do matching
school_id = 'SCHOOL_ID_-_NCES_ASSIGNED__PUBLIC_SCHOOL__LATEST_AVAILABLE_YEAR'
cc_crosswalk = cc_data_merged[['SCHOOL_NAME', 
                        school_id]].copy().drop_duplicates()


cc_crosswalk['name_tocompare_commonc'] = [replace_schooltype(one_school) for one_school in cc_crosswalk.SCHOOL_NAME]


## 2.2 Create name-nces id crosswalk using CRDC data

In [12]:
## try looking in CRDC--- create similar crosswalk; exact and non

crdc_largerpull = pd.read_csv("../../raw_input/dc/EducationDataPortal_03.08.2020_schools.csv")
crdc_crosswalk = crdc_largerpull[['school_name', 'ncessch']].copy().drop_duplicates()
crdc_crosswalk['school_name'] = crdc_crosswalk.school_name.str.upper()

## clean up name in similar way
crdc_crosswalk['name_tocompare_crdc'] = [replace_schooltype(one_school) 
                                 for one_school in crdc_crosswalk.school_name]

## 2.3 Clean school name in filings data

In [13]:
dc_filings['derived_schoolagainst'] = np.where(~dc_filings.against_dcps_name.isnull(),
                                              dc_filings.against_dcps_name,
                                        np.where(~dc_filings.against_charter_name.isnull(),
                                                dc_filings.against_charter_name,
                                         np.where(~dc_filings.stud_attendschool.isnull(),
                                                 dc_filings.stud_attendschool,
                                                 np.nan)))
dc_filings['school_against_cleaned_1'] = [process_schoolname(one_name) 
                                                for one_name in dc_filings.derived_schoolagainst]
dc_filings['school_against_cleaned'] = [replace_schooltype(one_name)
                                               for one_name in dc_filings.school_against_cleaned_1]
# dc_filings[dc_filings.derived_schoolagainst.isnull()] - to return to



## 2.4 Try exact matches

Hierarchical where we first try CRDC, then try common core, since the former has more granular school ids than latter (which codes all DCPS schools to same id)

In [14]:
## first, try exact match with crdc
dc_filings_exact = pd.merge(dc_filings[~dc_filings.derived_schoolagainst.isnull()].copy(),
                           crdc_crosswalk,
                            left_on = 'school_against_cleaned',
                            right_on = 'name_tocompare_crdc',
                           how = "left",
                           indicator = "exactm_crdc")

## then, try exact match of those with ccd
dc_filings_exact_c = pd.merge(dc_filings_exact,
                           cc_crosswalk,
                            left_on = 'school_against_cleaned',
                            right_on = 'name_tocompare_commonc',
                           how = "left",
                           indicator = "exactm_ccd",
                        suffixes = ["_crdc", "_ccd"])

## code categories
dc_filings_exact_c['match_cat'] = np.where((dc_filings_exact_c.exactm_crdc == "both") & 
                                           (dc_filings_exact_c.exactm_ccd == "both"),
                                        "Both exact",
                                np.where((dc_filings_exact_c.exactm_ccd == "both"), 
                                        "CCD exact only",
                                np.where(dc_filings_exact_c.exactm_crdc == "both",
                                        "CRDC exact only",
                                        "Neither exact")))

dc_filings_exact_c.match_cat.value_counts(normalize = True)


Both exact        0.498
Neither exact     0.450
CRDC exact only   0.038
CCD exact only    0.014
Name: match_cat, dtype: float64

In [15]:
## separate into three categories:
## 1. exact match
## 2. not exact match but needs it
## 3. not applicable
filings_exactm = dc_filings_exact_c.loc[dc_filings_exact_c.match_cat != "Neither exact"].copy()
filings_needfuzzy = dc_filings_exact_c.loc[(~dc_filings_exact_c.school_against_cleaned.isnull()) &
                                   (dc_filings_exact_c.match_cat ==  "Neither exact") &
                                    (~dc_filings_exact_c.derived_casetype_final.isin(['Against SEA', 'By LEA']))]
caseno_tomatch = filings_exactm.case_no.to_list() + filings_needfuzzy.case_no.to_list()
filings_other = dc_filings_exact_c[~dc_filings_exact_c.case_no.isin(caseno_tomatch)].copy()
"""{} cases exact; {} need fuzzy; {} other""".format(filings_exactm.shape[0],
                                                filings_needfuzzy.shape[0],
                                                filings_other.shape[0])

'2587 cases exact; 2090 need fuzzy; 25 other'

In [16]:
## for ones need fuzzy, subset to names only
filings_crosswalk = filings_needfuzzy[['school_against_cleaned']].drop_duplicates()
filings_crosswalk['id'] = filings_crosswalk.index+1
filings_crosswalk.shape


## write to intermediate --- first try crdc versus filings; then common core
crdc_crosswalk.to_csv("../../intermediate_objects/crdc_schoolnames_f22run.csv")
filings_crosswalk.to_csv("../../intermediate_objects/filings_names_f22run.csv")

(227, 2)

In [17]:
## Ran script: 03helper_fuzzymatch_nces.csv


# cc_crosswalk.to_csv("../../intermediate_objects/nces_schoolnames_f22run.csv")


# 3 merge in fuzzy match results

## 3.1 success via fuzzy match

In [29]:
## load results of fmatch
fm_crdc_filings = pd.read_csv("../../intermediate_objects/filings_fuzzymatch_crdc_fall22.csv")

## subset to 90 and above
fm_crdc_filings_keep = fm_crdc_filings[fm_crdc_filings.score >= 90].copy()

## write the kept ones for RA to check
fm_crdc_filings_keep['keep'] = 1
fm_crdc_filings_keep['school_new'] = ""
fm_crdc_filings_keep.sort_values(by = 'score', ascending = True).to_csv("../../filings_fuzzymatch_RAcheck.csv",
                                                                       index = False)




In [ ]:
## merge back using original name
filings_crosswalk_wmatch = pd.merge(filings_crosswalk, 
                                   fm_crdc_filings_keep[['original_name', 'matched_name', 'score']],
                                   left_on = 'school_against_cleaned',
                                   right_on = 'original_name',
                                   how = "left")

filings_crosswalk_wmatch['matched'] = filings_crosswalk_wmatch.score.notnull()


## separate into matched and nonmatched
filings_matchfuzzy = filings_crosswalk_wmatch[filings_crosswalk_wmatch.matched].copy()
filings_nonmatchfuzzy = filings_crosswalk_wmatch[~filings_crosswalk_wmatch.matched].sort_values(by = 'original_name')
filings_nonmatchfuzzy.to_csv("../../intermediate_objects/dc_snames_nonmatch_f22.csv",
                            index = False)

In [19]:
## merge in nces ID based on matched name
filings_crosswalk_wid = pd.merge(filings_crosswalk_wmatch[filings_crosswalk_wmatch.matched],
                                crdc_crosswalk.drop_duplicates(),
                                 left_on = 'matched_name',
                                 right_on = 'name_tocompare_crdc',
                                 how = "left").drop_duplicates()

## 3.2 manual match 


In [20]:
## read in manual matches
filings_manual = pd.read_csv("../../intermediate_objects/dc_snames_match_f22 - dc_snames_nonmatch_f22.csv")
filings_manual_matched = filings_manual[filings_manual.matched_name.notnull()].copy()
filings_crosswalk_wid_manual = pd.merge(filings_manual_matched,
                                crdc_crosswalk.drop_duplicates(),
                                 left_on = 'matched_name',
                                 right_on = 'name_tocompare_crdc',
                                 how = "left").drop_duplicates()



## look at onces not matched
filings_manual_notmatched = filings_manual[filings_manual.matched_name.isnull()].copy()

## write the results
#filings_towrite = filings_manual_notmatched[['school_against_cleaned', 'notes']]
#filings_towrite['school'] = filings_towrite.school_against_cleaned.str.replace(".*PG|-|•", "")
#filings_towrite['notes'] = np.where(filings_towrite.notes == "Many schools",
 #                                  "Multiple campuses", 
  #                                 filings_towrite.notes)
#filings_distinct = filings_towrite[['school', 'notes']].drop_duplicates(subset = ['school']).sort_values(by = 
                   #                     'school')
#filings_distinct = filings_distinct.fillna("")
#print(filings_distinct.to_latex(index = False))


In [21]:
## separate ones that we tried to match into two categories:
## (1) successfully matched or
## (2) not
matches_both = filings_crosswalk_wid_manual.school_against_cleaned.to_list() + \
                                    filings_crosswalk_wid.original_name.to_list()
filings_needfuzzy_matched = filings_needfuzzy[filings_needfuzzy.school_against_cleaned.isin(\
                                            matches_both)].copy()
filings_needfuzzy_notmatched = filings_needfuzzy[~filings_needfuzzy.school_against_cleaned.isin(\
                                            matches_both)].copy()


filings_needfuzzy_tomerge = filings_needfuzzy_matched[[col for col in filings_needfuzzy_matched.columns
                                             if col not in ["ncessch",
                                            "name_tocompare_crdc"]]].copy()

In [22]:
filings_needfuzzy_merged = pd.merge(filings_needfuzzy_tomerge,
                                   filings_crosswalk_wid[['name_tocompare_crdc',
                                                         'ncessch', 'original_name']].copy(),
                                   left_on = "school_against_cleaned",
                                   right_on = "original_name",
                                   how = "left",
                                   indicator = "fuzzym")



In [23]:
filings_needfuzzy_merged['match_cat'] = 'Fuzzy CRDC or manual'

In [26]:
## rowbind the two datasets:
## 1. match exact
## 2. match fuzzy
## for the match neither, trying the other source 
overlap_cols = set(filings_exactm.columns).intersection(filings_needfuzzy_merged.columns)
filings_exactfuzzy = pd.concat([filings_needfuzzy_merged[overlap_cols],
                               filings_exactm[overlap_cols]]).drop_duplicates()


## go back to filings and subset to the relevant school years - summarize match rates
## SYs: 2014-2015 to 2017-2018
## SY 2014 start date: https://dcps.dc.gov/sites/default/files/dc/sites/dcps/publication/attachments/REVISED%20School%20Year%202014-2015%20Full%20Calendar%20update.pdf
## 08-25-2014
sy_1415 = pd.to_datetime("2014-08-25")
sy_1718 = pd.to_datetime("2018-06-15")

dc_filings['in_date_range'] = np.where((dc_filings.derived_date_filed >= sy_1415) &
                              (dc_filings.derived_date_filed <= sy_1718), True, False)
filings_exactfuzzy['in_date_range'] = np.where((filings_exactfuzzy.derived_date_filed >= sy_1415) &
                              (filings_exactfuzzy.derived_date_filed <= sy_1718), True, False)

## check how many matched
filings_matched_daterange = filings_exactfuzzy[filings_exactfuzzy.in_date_range].copy()


## cols to keep
cols_keep = ['case_no', 'type_closingorder', 
            'ncessch', 'school_against_cleaned', 
            'name_tocompare_commonc', 'name_tocompare_crdc',
            'match_cat', 'SCHOOL_ID_-_NCES_ASSIGNED__PUBLIC_SCHOOL__LATEST_AVAILABLE_YEAR'] + [col for col in filings_matched_daterange.columns if 
                           "derived" in col]

NameError: name 'filings_daterange' is not defined

In [295]:
filings_matched_towrite = filings_matched_daterange[cols_keep].copy()
filings_matched_towrite.to_csv("../../intermediate_objects/cleaned_df/dc_filings.csv",
                              index = False) 
filings_matched_towrite.to_pickle("../../intermediate_objects/cleaned_df/dc_filings.pkl") 